<a href="https://colab.research.google.com/github/data04190/Capston_Design_Project/blob/main/crawling/stock_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import json
import pytz
import requests
import pandas as pd
from io import BytesIO
from datetime import timedelta
from datetime import datetime 


In [19]:
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/data/fullcode.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# 한국날짜 설정

def yesterday():
    tz = pytz.timezone('Asia/Seoul')  
    raw_date = datetime.now(tz)
    yesterday = (raw_date - timedelta(1)).strftime("%Y%m%d")   
    return yesterday

print(yesterday())

20220521


In [7]:
fullcode = pd.read_csv(data_path)
fullcode

,full_code,short_code,codeName,marketEngName,status
0,KR7060310000,060310,3S,KOSDAQ,상장
1,KR7095570008,095570,AJ네트웍스,KOSPI,상장
2,KR7006840003,006840,AK홀딩스,KOSPI,상장
3,KR7054620000,054620,APS홀딩스,KOSDAQ,상장
4,KR7265520007,265520,AP시스템,KOSDAQ,상장
...,...,...,...,...,...
2621,KR7003280005,003280,흥아해운,KOSPI,상장
2622,KR7037440005,037440,희림,KOSDAQ,상장
2623,KR7238490007,238490,힘스,KOSDAQ,상장
2624,KR7389260001,389260,대명에너지,KOSDAQ,상장


In [21]:
# 상장폐지 상태 제외시키기
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']
fullcode_filter

,full_code,short_code,codeName,marketEngName,status
0,KR7060310000,060310,3S,KOSDAQ,상장
1,KR7095570008,095570,AJ네트웍스,KOSPI,상장
2,KR7006840003,006840,AK홀딩스,KOSPI,상장
3,KR7054620000,054620,APS홀딩스,KOSDAQ,상장
4,KR7265520007,265520,AP시스템,KOSDAQ,상장
...,...,...,...,...,...
2621,KR7003280005,003280,흥아해운,KOSPI,상장
2622,KR7037440005,037440,희림,KOSDAQ,상장
2623,KR7238490007,238490,힘스,KOSDAQ,상장
2624,KR7389260001,389260,대명에너지,KOSDAQ,상장


In [22]:
# fullcode열을 list로 변경
fullcode_list = list(fullcode_filter['full_code'])
fullcode_list[:10]

['KR7060310000',
 'KR7095570008',
 'KR7006840003',
 'KR7054620000',
 'KR7265520007',
 'KR7211270004',
 'KR7027410000',
 'KR7282330000',
 'KR7032790008',
 'KR7138930003']

In [24]:
code = 'KR7060310000'

In [28]:
url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
headers = {'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',}

strtDd = '19000101'
endDd = yesterday()


data2 = { 'bld': 'dbms/MDC/STAT/issue/MDCSTAT23902',
'isuCd': code, 
'isuCd2': '',
'strtDd': strtDd,
'endDd': endDd,
'share': '1',
'money': '1',
'csvxls_isNo': 'false',}

r2 = requests.post(url2, data2, headers)
j2 = json.loads(r2.text)
df2 = pd.DataFrame(j2['output'])
df2



,TRD_DD,ISU_CD,ISU_NM,MKT_NM,SECUGRP_NM,TDD_CLSPRC,FLUC_TP_CD,CMPPRVDD_PRC,FLUC_RT,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL,MKTCAP
0,2022/05/20,060310,3S,KOSDAQ,주권,"3,490",1,360,11.50,"3,190","3,980","3,110","19,405,093","71,269,734,185","161,487,580,370"
1,2022/05/19,060310,3S,KOSDAQ,주권,"3,130",2,-10,-0.32,"3,085","3,140","3,070","193,365","599,842,595","144,829,835,690"
2,2022/05/18,060310,3S,KOSDAQ,주권,"3,140",1,70,2.28,"3,110","3,230","3,085","455,257","1,427,070,480","145,292,550,820"
3,2022/05/17,060310,3S,KOSDAQ,주권,"3,070",1,50,1.66,"3,020","3,480","2,950","3,464,672","11,263,417,785","142,053,544,910"
4,2022/05/16,060310,3S,KOSDAQ,주권,"3,020",1,30,1.00,"2,995","3,080","2,990","100,578","305,749,185","139,739,969,260"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4960,2002/04/29,060310,삼에스코리아,KOSDAQ,주권,"6,780",5,-920,-11.95,"6,800","7,200","6,780","139,388","951,796,930","16,272,000,000"
4961,2002/04/26,060310,삼에스코리아,KOSDAQ,주권,"7,700",5,"-1,050",-12.00,"8,550","8,750","7,700","176,111","1,405,023,020","18,480,000,000"
4962,2002/04/25,060310,삼에스코리아,KOSDAQ,주권,"8,750",1,240,2.82,"9,530","9,530","7,490","768,702","6,815,533,300","21,000,000,000"
4963,2002/04/24,060310,삼에스코리아,KOSDAQ,주권,"8,510",4,910,11.97,"8,370","8,510","7,750","140,723","1,169,025,100","20,424,000,000"


In [29]:
data3 = { 'bld': 'dbms/MDC/STAT/standard/MDCSTAT03502',
'isuCd': code,
'isuCd2': code,
'strtDd': strtDd,
'endDd': endDd,
'searchType': '2',
'mktId': 'ALL',
'csvxls_isNo': 'false',}

r3 = requests.post(url2, data3, headers)
j3 = json.loads(r3.text)
df3 = pd.DataFrame(j3['output'])
df3

,TRD_DD,TDD_CLSPRC,FLUC_TP_CD,CMPPREVDD_PRC,FLUC_RT,EPS,PER,FWD_EPS,FWD_PER,BPS,PBR,DPS,DVD_YLD
0,2022/05/20,"3,490",1,360,11.50,16,218.13,-,-,792,4.41,0,0.00
1,2022/05/19,"3,130",2,-10,-0.32,16,195.63,-,-,792,3.95,0,0.00
2,2022/05/18,"3,140",1,70,2.28,16,196.25,-,-,792,3.96,0,0.00
3,2022/05/17,"3,070",1,50,1.66,16,191.88,-,-,792,3.88,0,0.00
4,2022/05/16,"3,020",1,30,1.00,16,188.75,-,-,792,3.81,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4960,2002/04/29,"6,780",5,-920,-11.95,617,10.99,-,-,-,-,-,-
4961,2002/04/26,"7,700",5,"-1,050",-12.00,617,12.48,-,-,-,-,-,-
4962,2002/04/25,"8,750",1,240,2.82,617,14.18,-,-,-,-,-,-
4963,2002/04/24,"8,510",4,910,11.97,617,13.79,-,-,-,-,-,-


In [30]:
if len(df2.columns) > 0: #df2에 column이 존재하면 (df가 비어서 에러나는 경우 방지)
    df2_ = df2[['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP', 'ACC_TRDVOL']]
else: # df가 비었을때 column 생성.
    df2_ = pd.DataFrame(index = range(0,len(df3)),
                        columns = ['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP', 'ACC_TRDVOL'])

if len(df3.columns) > 0:
    df3_ = df3[['EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD']]
else:
    df3_ = pd.DataFrame(index = range(0,len(df2)),
                        columns = ['EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD'])

result = pd.concat([df2_, df3_], axis = 1)
result

,TRD_DD,ISU_CD,ISU_NM,TDD_CLSPRC,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,MKTCAP,ACC_TRDVOL,EPS,PER,BPS,PBR,DPS,DVD_YLD
0,2022/05/20,060310,3S,"3,490","3,190","3,980","3,110","161,487,580,370","19,405,093",16,218.13,792,4.41,0,0.00
1,2022/05/19,060310,3S,"3,130","3,085","3,140","3,070","144,829,835,690","193,365",16,195.63,792,3.95,0,0.00
2,2022/05/18,060310,3S,"3,140","3,110","3,230","3,085","145,292,550,820","455,257",16,196.25,792,3.96,0,0.00
3,2022/05/17,060310,3S,"3,070","3,020","3,480","2,950","142,053,544,910","3,464,672",16,191.88,792,3.88,0,0.00
4,2022/05/16,060310,3S,"3,020","2,995","3,080","2,990","139,739,969,260","100,578",16,188.75,792,3.81,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4960,2002/04/29,060310,삼에스코리아,"6,780","6,800","7,200","6,780","16,272,000,000","139,388",617,10.99,-,-,-,-
4961,2002/04/26,060310,삼에스코리아,"7,700","8,550","8,750","7,700","18,480,000,000","176,111",617,12.48,-,-,-,-
4962,2002/04/25,060310,삼에스코리아,"8,750","9,530","9,530","7,490","21,000,000,000","768,702",617,14.18,-,-,-,-
4963,2002/04/24,060310,삼에스코리아,"8,510","8,370","8,510","7,750","20,424,000,000","140,723",617,13.79,-,-,-,-


In [37]:
result.to_csv('/content/drive/MyDrive/data/'+code+".csv")